In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -U sentence-transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 77 kB 7.0 MB/s 
     |████████████████████████████████| 895 kB 70.6 MB/s 
     |████████████████████████████████| 6.5 MB 58.9 MB/s 
     |████████████████████████████████| 596 kB 61.1 MB/s 
     |████████████████████████████████| 79 kB 9.2 MB/s 
     |████████████████████████████████| 1.2 MB 63.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=432a3171d2b1a5bd94e1ff82ff88f1c1381ff7f486eea6937728498066c0e196
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
% cd /content/drive/MyDrive

In [ ]:
import torch
import torch.nn as nn
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import pickle
import json
import os
import pandas as pd
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatiser = WordNetLemmatizer()
nltk.download('wordnet')

In [ ]:
# don't convert to lowercase as using bert base cased (case is important)
def preprocess(text):
    res = re.sub(r'[^\w\s]', '',text) # Remove punctuations
    res = " ".join(res.split())       # Remove whitespaces
    tokens = word_tokenize(res) 
    new_tokens = [lemmatiser.lemmatize(i) for i in tokens if not i in stop_words] # Remove stopwords and lemmatise
    res = ' '.join(new_tokens)
    return res



In [ ]:
f = open('all_texts.json')
# returns JSON object as a dictionary
all_texts = json.load(f)
print(len(all_texts))

9493


In [ ]:
for i in all_texts:
    all_texts[i] = preprocess(all_texts[i])

In [ ]:
def func(name, label):                                                          
  df = pd.read_csv(name)
  df.head()
  l1 = []
  l2 = []
  l3 = []
  for index,row in df.iterrows():
      id1,id2 = row['pair_id'].split('_')
      if id1 in all_texts and id2 in all_texts:
         l1.append(all_texts[id1])
         l2.append(all_texts[id2])
         l3.append((row[label]-1)/3)
  di = {'Text 1':l1, 'Text 2':l2, 'GT': l3}
  df = pd.DataFrame(di)
  train_examples=[]
  for i in range(len(df)):
    train_examples.append(InputExample(texts=[df["Text 1"][i], df["Text 2"][i]], label=float(df["GT"][i])))
  return(train_examples)

In [ ]:
# modify func to take in the gt for the rest labels(6) look into data.csv for label name (di['GT'])   

# train mbert, try xlm if it can perform     mbert<xlm-base<xlm-large

# xlm large 2e-4 

# at max 4 epoch

# 

In [ ]:
all_labels = ["Geography","Entities","Time","Narrative","Overall","Style","Tone"]
train_examples=func("/content/drive/MyDrive/data.csv", all_labels[6]) ## change lables accordingly

# For training



In [ ]:
def train(model_name,batch_size,lr,loss,epochs):
  model1 = CrossEncoder(model_name,num_labels=1)
  train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
  if(loss=="L1"):
    model1.fit(train_dataloader=train_dataloader,epochs=epochs,loss_fct = nn.L1Loss(),scheduler = "constantlr")
  elif(loss=="MSE"):
    model1.fit(train_dataloader=train_dataloader,epochs=epochs,loss_fct = nn.MSELoss(),scheduler = "constantlr")
  elif(loss=="CosineSimilarity"):
    model1.fit(train_dataloader=train_dataloader,epochs=epochs,loss_fct = nn.CosineSimilarity(dim=1, eps=1e-6),scheduler = "constantlr",optimizer_params= {'lr': lr})
  filename = "Weights/" + model_name+"_"+str(batch_size)+"_"+str(lr)+"_"+str(loss)+"_"+str(epochs)
  pickle.dump(model1, open(filename, 'wb'))

In [ ]:
def train_from_epoch(model_file_location, epochs_old ,model_name,batch_size,lr,loss, epochs):
  model1 = pickle.load(open(model_file_location, 'rb'))
  train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
  if(loss=="L1"):
    model1.fit(train_dataloader=train_dataloader,epochs=epochs,loss_fct = nn.L1Loss(),scheduler = "constantlr")
  elif(loss=="MSE"):
    model1.fit(train_dataloader=train_dataloader,epochs=epochs,loss_fct = nn.MSELoss(),scheduler = "constantlr")
  elif(loss=="CosineSimilarity"):
    model1.fit(train_dataloader=train_dataloader,epochs=epochs,loss_fct = nn.CosineSimilarity(dim=1, eps=1e-6),scheduler = "constantlr",optimizer_params= {'lr': lr})
  filename = "Weights/" + model_name+"_"+str(batch_size)+"_"+str(lr)+"_"+str(loss)+"_"+str(epochs+epochs_old)
  pickle.dump(model1, open(filename, 'wb'))

In [ ]:
train_from_epoch("/content/drive/MyDrive/Weights/bert-base-multilingual-cased_8_2e-05_MSE_1",1,"bert-base-multilingual-cased",8,2e-5,"MSE",1)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/618 [00:00<?, ?it/s]

In [ ]:
train_from_epoch("/content/drive/MyDrive/Weights/bert-base-multilingual-cased_8_2e-05_MSE_2",2,"bert-base-multilingual-cased",8,2e-5,"MSE",1)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/618 [00:00<?, ?it/s]

In [ ]:
train_from_epoch("/content/drive/MyDrive/Weights/bert-base-multilingual-cased_8_2e-05_MSE_3",3,"bert-base-multilingual-cased",8,2e-5,"MSE",1)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/618 [00:00<?, ?it/s]

In [ ]:
train_from_epoch("/content/drive/MyDrive/Weights/bert-base-multilingual-cased_8_2e-05_MSE_4",4,"bert-base-multilingual-cased",8,2e-5,"MSE",1)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/618 [00:00<?, ?it/s]

In [ ]:
train_from_epoch("/content/drive/MyDrive/Weights/bert-base-multilingual-cased_8_2e-05_MSE_5",5,"bert-base-multilingual-cased",8,2e-5,"MSE",1)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/618 [00:00<?, ?it/s]

In [ ]:
train("bert-base-multilingual-cased",8,2e-5,"MSE",1)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/618 [00:00<?, ?it/s]

For Validation

In [ ]:
with open("eval_texts.json", "r") as fp:
  eval_texts = json.load(fp)
for i in eval_texts:
    eval_texts[i] = preprocess(eval_texts[i])
def evaluate(name, label):
  df = pd.read_csv(name)
  l1 = []
  l2 = []
  l3 = []
  for index,row in df.iterrows():
      id1,id2 = row['pair_id'].split('_')
      if id1 in eval_texts and id2 in eval_texts:
         l1.append(eval_texts[id1])
         l2.append(eval_texts[id2])
         l3.append((row[label]-1)/3)
  di = {'Text 1':l1, 'Text 2':l2, 'GT': l3}
  df = pd.DataFrame(di)
  test_examples=[]
  score=[]
  for i in range(len(df)):
    test_examples.append([df["Text 1"][i], df["Text 2"][i]])
    score.append(df["GT"][i])
    
  return(test_examples,score)

In [ ]:
all_labels = ["GEO","ENT","TIME","NAR","Overall","STYLE","TONE"]
data,score=evaluate("final_evaluation_data.csv", all_labels[6])
model1=pickle.load(open("/content/drive/MyDrive/Weights/bert-base-multilingual-cased_8_2e-05_MSE_1", 'rb'))
scores_predicted=model1.predict(data)
print(scores_predicted)
print(score)

x=torch.Tensor([scores_predicted,score])
coeff=torch.corrcoef(x)
print(coeff)

[0.53775996 0.5374415  0.59966224 ... 0.5706682  0.55733526 0.57402277]
[0.16666666666666666, 0.5, 0.8888888888888901, 0.3333333333333333, 0.5, 0.11111111111110998, 0.8333333333333334, 0.16666666666666666, 0.16666666666666666, 0.8888888888888901, 0.5555555555555567, 0.3333333333333333, 0.3333333333333333, 0.8888888888888901, 0.0, 0.16666666666666666, 0.6666666666666666, 0.0, 0.4444444444444433, 0.0, 0.0, 0.11111111111110998, 0.6666666666666666, 0.0, 0.0, 0.0, 0.7777777777777767, 0.8888888888888901, 0.11111111111110998, 0.6666666666666666, 0.5555555555555567, 0.22222222222222335, 0.16666666666666666, 0.0, 0.8333333333333334, 0.6666666666666666, 0.22222222222222335, 0.11111111111110998, 0.0, 0.0, 1.0, 0.3333333333333333, 0.0, 0.0, 0.6666666666666666, 0.0, 0.5, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.8333333333333334, 0.6666666666666666, 0.16666666666666666, 0.6666666666666666, 0.0, 1.0, 0.16666666666666666, 0.0, 0.3333333333333333, 0.11111111111110998, 0.11111111111110998, 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  


In [ ]:
all_labels = ["GEO","ENT","TIME","NAR","Overall","STYLE","TONE"]
data,score=evaluate("final_evaluation_data.csv", all_labels[6])
model1=pickle.load(open("/content/drive/MyDrive/Weights/bert-base-multilingual-cased_8_2e-05_MSE_2", 'rb'))
scores_predicted=model1.predict(data)
print(scores_predicted)
print(score)

x=torch.Tensor([scores_predicted,score])
coeff=torch.corrcoef(x)
print(coeff)

[0.5505499  0.5573765  0.60447204 ... 0.5854395  0.56425524 0.5646133 ]
[0.16666666666666666, 0.5, 0.8888888888888901, 0.3333333333333333, 0.5, 0.11111111111110998, 0.8333333333333334, 0.16666666666666666, 0.16666666666666666, 0.8888888888888901, 0.5555555555555567, 0.3333333333333333, 0.3333333333333333, 0.8888888888888901, 0.0, 0.16666666666666666, 0.6666666666666666, 0.0, 0.4444444444444433, 0.0, 0.0, 0.11111111111110998, 0.6666666666666666, 0.0, 0.0, 0.0, 0.7777777777777767, 0.8888888888888901, 0.11111111111110998, 0.6666666666666666, 0.5555555555555567, 0.22222222222222335, 0.16666666666666666, 0.0, 0.8333333333333334, 0.6666666666666666, 0.22222222222222335, 0.11111111111110998, 0.0, 0.0, 1.0, 0.3333333333333333, 0.0, 0.0, 0.6666666666666666, 0.0, 0.5, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.8333333333333334, 0.6666666666666666, 0.16666666666666666, 0.6666666666666666, 0.0, 1.0, 0.16666666666666666, 0.0, 0.3333333333333333, 0.11111111111110998, 0.11111111111110998, 

In [ ]:
all_labels = ["GEO","ENT","TIME","NAR","Overall","STYLE","TONE"]
data,score=evaluate("final_evaluation_data.csv", all_labels[6])
model1=pickle.load(open("/content/drive/MyDrive/Weights/bert-base-multilingual-cased_8_2e-05_MSE_3", 'rb'))
scores_predicted=model1.predict(data)
print(scores_predicted)
print(score)

x=torch.Tensor([scores_predicted,score])
coeff=torch.corrcoef(x)
print(coeff)

[0.51050943 0.50201404 0.5499217  ... 0.60554194 0.5215611  0.57594097]
[0.16666666666666666, 0.5, 0.8888888888888901, 0.3333333333333333, 0.5, 0.11111111111110998, 0.8333333333333334, 0.16666666666666666, 0.16666666666666666, 0.8888888888888901, 0.5555555555555567, 0.3333333333333333, 0.3333333333333333, 0.8888888888888901, 0.0, 0.16666666666666666, 0.6666666666666666, 0.0, 0.4444444444444433, 0.0, 0.0, 0.11111111111110998, 0.6666666666666666, 0.0, 0.0, 0.0, 0.7777777777777767, 0.8888888888888901, 0.11111111111110998, 0.6666666666666666, 0.5555555555555567, 0.22222222222222335, 0.16666666666666666, 0.0, 0.8333333333333334, 0.6666666666666666, 0.22222222222222335, 0.11111111111110998, 0.0, 0.0, 1.0, 0.3333333333333333, 0.0, 0.0, 0.6666666666666666, 0.0, 0.5, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.8333333333333334, 0.6666666666666666, 0.16666666666666666, 0.6666666666666666, 0.0, 1.0, 0.16666666666666666, 0.0, 0.3333333333333333, 0.11111111111110998, 0.11111111111110998, 

In [ ]:
all_labels = ["GEO","ENT","TIME","NAR","Overall","STYLE","TONE"]
data,score=evaluate("final_evaluation_data.csv", all_labels[6])
model1=pickle.load(open("/content/drive/MyDrive/Weights/bert-base-multilingual-cased_8_2e-05_MSE_4", 'rb'))
scores_predicted=model1.predict(data)
print(scores_predicted)
print(score)

x=torch.Tensor([scores_predicted,score])
coeff=torch.corrcoef(x)
print(coeff)

[0.52002716 0.5291783  0.57743996 ... 0.59541416 0.54811734 0.53891367]
[0.16666666666666666, 0.5, 0.8888888888888901, 0.3333333333333333, 0.5, 0.11111111111110998, 0.8333333333333334, 0.16666666666666666, 0.16666666666666666, 0.8888888888888901, 0.5555555555555567, 0.3333333333333333, 0.3333333333333333, 0.8888888888888901, 0.0, 0.16666666666666666, 0.6666666666666666, 0.0, 0.4444444444444433, 0.0, 0.0, 0.11111111111110998, 0.6666666666666666, 0.0, 0.0, 0.0, 0.7777777777777767, 0.8888888888888901, 0.11111111111110998, 0.6666666666666666, 0.5555555555555567, 0.22222222222222335, 0.16666666666666666, 0.0, 0.8333333333333334, 0.6666666666666666, 0.22222222222222335, 0.11111111111110998, 0.0, 0.0, 1.0, 0.3333333333333333, 0.0, 0.0, 0.6666666666666666, 0.0, 0.5, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.8333333333333334, 0.6666666666666666, 0.16666666666666666, 0.6666666666666666, 0.0, 1.0, 0.16666666666666666, 0.0, 0.3333333333333333, 0.11111111111110998, 0.11111111111110998, 

In [ ]:
all_labels = ["GEO","ENT","TIME","NAR","Overall","STYLE","TONE"]
data,score=evaluate("final_evaluation_data.csv", all_labels[6])
model1=pickle.load(open("/content/drive/MyDrive/Weights/bert-base-multilingual-cased_8_2e-05_MSE_5", 'rb'))
scores_predicted=model1.predict(data)
print(scores_predicted)
print(score)

x=torch.Tensor([scores_predicted,score])
coeff=torch.corrcoef(x)
print(coeff)

[0.55436194 0.6199094  0.5967642  ... 0.57784206 0.5188694  0.60564053]
[0.16666666666666666, 0.5, 0.8888888888888901, 0.3333333333333333, 0.5, 0.11111111111110998, 0.8333333333333334, 0.16666666666666666, 0.16666666666666666, 0.8888888888888901, 0.5555555555555567, 0.3333333333333333, 0.3333333333333333, 0.8888888888888901, 0.0, 0.16666666666666666, 0.6666666666666666, 0.0, 0.4444444444444433, 0.0, 0.0, 0.11111111111110998, 0.6666666666666666, 0.0, 0.0, 0.0, 0.7777777777777767, 0.8888888888888901, 0.11111111111110998, 0.6666666666666666, 0.5555555555555567, 0.22222222222222335, 0.16666666666666666, 0.0, 0.8333333333333334, 0.6666666666666666, 0.22222222222222335, 0.11111111111110998, 0.0, 0.0, 1.0, 0.3333333333333333, 0.0, 0.0, 0.6666666666666666, 0.0, 0.5, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.8333333333333334, 0.6666666666666666, 0.16666666666666666, 0.6666666666666666, 0.0, 1.0, 0.16666666666666666, 0.0, 0.3333333333333333, 0.11111111111110998, 0.11111111111110998, 

In [ ]:
all_labels = ["GEO","ENT","TIME","NAR","Overall","STYLE","TONE"]
data,score=evaluate("final_evaluation_data.csv", all_labels[6])
model1=pickle.load(open("/content/drive/MyDrive/Weights/bert-base-multilingual-cased_8_2e-05_MSE_6", 'rb'))
scores_predicted=model1.predict(data)
print(scores_predicted)
print(score)

x=torch.Tensor([scores_predicted,score])
coeff=torch.corrcoef(x)
print(coeff)

[0.54157954 0.5786833  0.6201427  ... 0.5901781  0.52858025 0.61069137]
[0.16666666666666666, 0.5, 0.8888888888888901, 0.3333333333333333, 0.5, 0.11111111111110998, 0.8333333333333334, 0.16666666666666666, 0.16666666666666666, 0.8888888888888901, 0.5555555555555567, 0.3333333333333333, 0.3333333333333333, 0.8888888888888901, 0.0, 0.16666666666666666, 0.6666666666666666, 0.0, 0.4444444444444433, 0.0, 0.0, 0.11111111111110998, 0.6666666666666666, 0.0, 0.0, 0.0, 0.7777777777777767, 0.8888888888888901, 0.11111111111110998, 0.6666666666666666, 0.5555555555555567, 0.22222222222222335, 0.16666666666666666, 0.0, 0.8333333333333334, 0.6666666666666666, 0.22222222222222335, 0.11111111111110998, 0.0, 0.0, 1.0, 0.3333333333333333, 0.0, 0.0, 0.6666666666666666, 0.0, 0.5, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.8333333333333334, 0.6666666666666666, 0.16666666666666666, 0.6666666666666666, 0.0, 1.0, 0.16666666666666666, 0.0, 0.3333333333333333, 0.11111111111110998, 0.11111111111110998, 

#Reported Correlation coefficients of different features

**GEOGRAPHY - L1**
---

*   "bert-base-multilingual-cased",1,2e-5,"L1",1   ->   0.5064
*   "bert-base-multilingual-cased",4,2e-5,"L1",2   ->   0.5140
*   "bert-base-multilingual-cased",4,2e-5,"L1",3   ->   0.5727
*   "bert-base-multilingual-cased",4,2e-5,"L1",4   ->   0.5439
*   "bert-base-multilingual-cased",4,2e-5,"L1",5   ->   0.5590
*   "bert-base-multilingual-cased",4,2e-5,"L1",6   ->   0.5371

**GEOGRAPHY - MSE**
---

*   "bert-base-multilingual-cased",8,2e-5,"MSE",1   ->   0.5722
*   "bert-base-multilingual-cased",8,2e-5,"MSE",2   ->   0.5795
*   "bert-base-multilingual-cased",8,2e-5,"MSE",3   ->   0.5829
*   "bert-base-multilingual-cased",8,2e-5,"MSE",4   ->   0.5775
*   "bert-base-multilingual-cased",8,2e-5,"MSE",5   ->   0.5776
*   "bert-base-multilingual-cased",8,2e-5,"MSE",6   ->   0.5875

**Entities - MSE**
---

*   "bert-base-multilingual-cased",8,2e-5,"MSE",1   ->  0.7300 
*   "bert-base-multilingual-cased",8,2e-5,"MSE",2   ->  0.7308
*   "bert-base-multilingual-cased",8,2e-5,"MSE",3   ->  0.7256
*   "bert-base-multilingual-cased",8,2e-5,"MSE",4   ->  0.7266
*   "bert-base-multilingual-cased",8,2e-5,"MSE",5   ->  0.7103
*   "bert-base-multilingual-cased",8,2e-5,"MSE",6   ->  0.7270


**Time - MSE**
---

*   "bert-base-multilingual-cased",8,2e-5,"MSE",1   ->  0.4787
*   "bert-base-multilingual-cased",8,2e-5,"MSE",2   ->  0.4767
*   "bert-base-multilingual-cased",8,2e-5,"MSE",3   ->  0.4420
*   "bert-base-multilingual-cased",8,2e-5,"MSE",4   ->  0.4558
*   "bert-base-multilingual-cased",8,2e-5,"MSE",5   ->  0.4644
*   "bert-base-multilingual-cased",8,2e-5,"MSE",6   ->  0.4848 

**Narrative - MSE**
---

*   "bert-base-multilingual-cased",8,2e-5,"MSE",1   ->  0.7026
*   "bert-base-multilingual-cased",8,2e-5,"MSE",2   ->  0.7022
*   "bert-base-multilingual-cased",8,2e-5,"MSE",3   ->  0.7144
*   "bert-base-multilingual-cased",8,2e-5,"MSE",4   ->  0.6902
*   "bert-base-multilingual-cased",8,2e-5,"MSE",5   ->  0.6961
*   "bert-base-multilingual-cased",8,2e-5,"MSE",6   ->  0.7055

**Style - MSE**
---

*   "bert-base-multilingual-cased",8,2e-5,"MSE",1   ->  
*   "bert-base-multilingual-cased",8,2e-5,"MSE",2   ->  
*   "bert-base-multilingual-cased",8,2e-5,"MSE",3   ->  
*   "bert-base-multilingual-cased",8,2e-5,"MSE",4   ->  
*   "bert-base-multilingual-cased",8,2e-5,"MSE",5   ->  
*   "bert-base-multilingual-cased",8,2e-5,"MSE",6   ->   

**Tone - MSE**
---

*   "bert-base-multilingual-cased",8,2e-5,"MSE",1   ->  0.4901
*   "bert-base-multilingual-cased",8,2e-5,"MSE",2   ->  0.4844
*   "bert-base-multilingual-cased",8,2e-5,"MSE",3   ->  0.4766
*   "bert-base-multilingual-cased",8,2e-5,"MSE",4   ->  0.4618
*   "bert-base-multilingual-cased",8,2e-5,"MSE",5   ->  0.4329
*   "bert-base-multilingual-cased",8,2e-5,"MSE",6   ->  0.4453


